# 0.라이브러리

In [ ]:
import requests
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
import folium
from folium.plugins import MiniMap

import time
import datetime

from collections import OrderedDict

# 1. 키워드

## 1. 1 함수 정의

In [ ]:
##카카오 API
def whole_region(keyword, start_x,start_y,end_x,end_y):
    #print(start_x,start_y,end_x,end_y)
    page_num = 1
    # 데이터가 담길 리스트
    all_data_list = []
    
    while(1):
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
        params = {'query': keyword,'page': page_num,
                'rect': f'{start_x},{start_y},{end_x},{end_y}'}
        headers = {'Authorization':'KakaoAK 57a426ba3fb221a28bbb05da0282184d'}
        resp = requests.get(url, params=params, headers=headers)
    
        search_count = resp.json()['meta']['total_count']
        #print('총 개수',search_count)
    
        if search_count > 45:
            #print('좌표 4등분')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            ## 4등분 중 왼쪽 아래
            all_data_list.extend(whole_region(keyword, start_x,start_y,dividing_x,dividing_y))
            ## 4등분 중 오른쪽 아래
            all_data_list.extend(whole_region(keyword, dividing_x,start_y,end_x,dividing_y))
            ## 4등분 중 왼쪽 위
            all_data_list.extend(whole_region(keyword, start_x,dividing_y,dividing_x,end_y))
            ## 4등분 중 오른쪽 위
            all_data_list.extend(whole_region(keyword, dividing_x,dividing_y,end_x,end_y))
            return all_data_list

        else:
            if resp.json()['meta']['is_end']:
                all_data_list.extend(resp.json()['documents'])
                return all_data_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                #print('다음페이지')
                page_num += 1
                all_data_list.extend(resp.json()['documents'])

In [ ]:
def overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y):
    # 최종 데이터가 담길 리스트
    overlapped_result = []
    
    # 지도를 사각형으로 나누면서 데이터 받아옴
    for i in range(1,num_x+1): ## 1,10
        end_x = start_x + next_x
        initial_start_y = start_y
        for j in range(1,num_y+1): ## 1,6
            end_y = initial_start_y + next_y
            each_result= whole_region(keyword, start_x,initial_start_y,end_x,end_y)
            overlapped_result.extend(each_result)
            initial_start_y = end_y
        start_x = end_x
    return overlapped_result

In [ ]:
def make_map(dfs):
    # 지도 생성하기
    m = folium.Map(location=[36.587496, 127.345869], # 기준좌표: 세종어딘가
                    zoom_start=12)

    # 미니맵 추가
    minimap = MiniMap()
    m.add_child(minimap)

    # 마커
    for i in range(len(dfs)):
        folium.Marker([df['Y'][i],df['X'][i]], # 세종 어딘가
                tooltip=dfs['name'][i]
                ).add_to(m)
    return m

In [ ]:
def crawling_1(keywords,csv_name):
    # 시작 x 좌표 및 증가값
    keyword = keywords
    start_x = 127.13
    start_y = 36.41
    next_x = 0.01
    next_y = 0.01
    num_x = 27
    num_y = 33
    
    overlapped_result = overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y)

    # 최종 데이터가 담긴 리스트 중복값 제거
    results = list(map(dict, OrderedDict.fromkeys(tuple(sorted(d.items())) for d in overlapped_result)))
    X = []
    Y = []
    names = []
    road_address = []
    ID = []
    address_name = []

    for place in results:
    
        X.append(float(place['x']))
        Y.append(float(place['y']))
        names.append(place['place_name'])
        road_address.append(place['road_address_name'])
        ID.append(place['id'])
        address_name.append(place['address_name'])
    
        ar = np.array([ID,names, X, Y, road_address,address_name]).T 
        df = pd.DataFrame(ar, columns = ['ID','name', 'X', 'Y','road_address','address_name']) # 수정
    print('total_reuslt_number = ',len(df))

    df.to_csv('%s.csv'%csv_name)

## 1.2. 크롤링

In [ ]:
crawling_1("주차장","parking")
crawling_1("편의점","convenience")
crawling_1("아파트","apt")
crawling_1("오피스텔","office")
crawling_1("빌라","villa")
crawling_1("주택","house")

total_reuslt_number =  457
total_reuslt_number =  494
total_reuslt_number =  800
total_reuslt_number =  559
total_reuslt_number =  612
total_reuslt_number =  643


# 2. 카테고리

## 2.1.함수 정의

In [ ]:
##카카오 API
def whole_region(category, start_x,start_y,end_x,end_y):
    page_num = 1
    # 데이터가 담길 리스트
    all_data_list = []
    
    while(1):
        url = 'https://dapi.kakao.com/v2/local/search/category.json' 
        params = {'category_group_code': category,'page': page_num,
                'rect': f'{start_x},{start_y},{end_x},{end_y}'}
        headers = {'Authorization':'KakaoAK 57a426ba3fb221a28bbb05da0282184d'}
        resp = requests.get(url, params=params, headers=headers)
    
        search_count = resp.json()['meta']['total_count']
        #print('총 개수',search_count)
    
        if search_count > 45:
            #print('좌표 4등분')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            ## 4등분 중 왼쪽 아래
            all_data_list.extend(whole_region(category, start_x,start_y,dividing_x,dividing_y))#
            ## 4등분 중 오른쪽 아래
            all_data_list.extend(whole_region(category, dividing_x,start_y,end_x,dividing_y))#
            ## 4등분 중 왼쪽 위
            all_data_list.extend(whole_region(category, start_x,dividing_y,dividing_x,end_y))#
            ## 4등분 중 오른쪽 위
            all_data_list.extend(whole_region(category, dividing_x,dividing_y,end_x,end_y))#
            return all_data_list

        else:
            if resp.json()['meta']['is_end']:
                all_data_list.extend(resp.json()['documents'])
                return all_data_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                #print('다음페이지')
                page_num += 1
                all_data_list.extend(resp.json()['documents'])

In [ ]:
def overlapped_data(category, start_x, start_y, next_x, next_y, num_x, num_y):#
    # 최종 데이터가 담길 리스트
    overlapped_result = []
    
    # 지도를 사각형으로 나누면서 데이터 받아옴
    for i in range(1,num_x+1): ## 1,10
        end_x = start_x + next_x
        initial_start_y = start_y
        for j in range(1,num_y+1): ## 1,6
            end_y = initial_start_y + next_y
            each_result= whole_region(category, start_x,initial_start_y,end_x,end_y)#
            overlapped_result.extend(each_result)
            initial_start_y = end_y
        start_x = end_x
    return overlapped_result

In [ ]:
def make_map(dfs):
    # 지도 생성하기
    m = folium.Map(location=[36.587496, 127.345869], # 기준좌표: 세종어딘가
                    zoom_start=12)

    # 미니맵 추가
    minimap = MiniMap()
    m.add_child(minimap)

    # 마커
    for i in range(len(dfs)):
        folium.Marker([df['Y'][i],df['X'][i]], # 세종 어딘가
                tooltip=dfs['name'][i]
                ).add_to(m)
    return m

In [ ]:
def crawling_2(categorys,csv_name):
    # 시작 x 좌표 및 증가값
    category = categorys
    start_x = 127.13
    start_y = 36.41
    next_x = 0.01
    next_y = 0.01
    num_x = 27
    num_y = 33

    overlapped_result = overlapped_data(category, start_x, start_y, next_x, next_y, num_x, num_y)

    # 최종 데이터가 담긴 리스트 중복값 제거
    results = list(map(dict, OrderedDict.fromkeys(tuple(sorted(d.items())) for d in overlapped_result)))
    X = []
    Y = []
    names = []
    ID = []
    address_name = []
    Category_code = []

    for place in results:
    
        X.append(float(place['x']))
        Y.append(float(place['y']))
        names.append(place['place_name'])
        ID.append(place['id'])
        address_name.append(place['address_name'])
        Category_code.append(place['category_group_code'])
    
        ar = np.array([ID, Category_code, names, X, Y, address_name]).T
        df = pd.DataFrame(ar, columns = ['ID','Category_code','name', 'X', 'Y','address_name'])
    print('total_reuslt_number = ',len(df))

    df.to_csv('%s.csv'%csv_name)

## 2.2.크롤링

In [ ]:
crawling_2("FD6","eat")
crawling_2("CE7","cafe")

total_reuslt_number =  6007
total_reuslt_number =  1216
